# Convolutional Neural Net Tutorial (tensorflow)

원문: https://www.kaggle.com/itratrahman/convolutional-neural-net-tutorial-tensorflow  
해당 Notebook의 한국어 해석본입니다. 주석을 달아서 해석을 했습니다.

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import time
t1 = time.time()
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import math
import random
import pandas as pd
import numpy as np
import cv2
from sklearn.preprocessing import MinMaxScaler
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline

## 데이터 로드 및 데이터 확인 (Load and Inspect the Data)

In [ ]:
#재현이 가능하도록 랜덤 시드 설정(NumPy, TensorFlow)
np_random_seed = 97
tf_rand_seed = 82
np.random.seed(np_random_seed)

In [ ]:
#test.json파일과 train.json이 .7z 파일로 압축되어 있어서 이를 압축해제하기 위한 library py7zr 설치 후 압축해제 실행.
!pip install py7zr
import py7zr
import os

if not os.path.exists('/kaggle/train/') :
    os.makedirs('/kaggle/train/')

if not os.path.exists('/kaggle/test/') :
    os.makedirs('/kaggle/test/')

with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/train.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/train")

with py7zr.SevenZipFile("/kaggle/input/statoil-iceberg-classifier-challenge/test.json.7z", 'r') as archive:
    archive.extractall(path="/kaggle/test")

for dirname, _, filenames in os.walk('/kaggle'): 
    for filename in filenames: 
        print(os.path.join(dirname, filename))

In [ ]:
train = pd.read_json('/kaggle/train/data/processed/train.json')
test = pd.read_json('/kaggle/test/data/processed/test.json')

In [ ]:
train.head(5)

In [ ]:
test.head(5)

In [ ]:
#train, test set의 크기 확인
print('Shape of train set:', train.shape)
print('Shape of test set:', test.shape)

In [ ]:
#train set의 band_1, band_2 feature의 크기 확인
print("Shape of band 1:",  np.shape(train.band_1.iloc[0]))
print("Shape of band 2:",  np.shape(train.band_2.iloc[0]))

In [ ]:
#train set의 band_1, band_2 feature의 data type 확인
print("Type of band 1:",  type(train.band_1.iloc[0]))
print("Type of band 2:",  type(train.band_2.iloc[0]))

## 2. Feature Engineering

### 2.1 Train Set에 대한 FE

In [ ]:
#train set의 inc_angle feature에서 결측치(na값)을 결측치가 없는 값들의 mean값으로 대치한다.
train[train['inc_angle']=='na'] = train[train['inc_angle']!='na']['inc_angle'].mean()

In [ ]:
#inc_angle feature는 각도(0~360)으로 되어있는데, 이를 라디안 값으로 변환한다.
train['inc_angle'] = train['inc_angle'].apply(lambda x: math.radians(x))

In [ ]:
train.inc_angle.head()
#밑을 보면 inc_angle 값이 라디안 값으로 변환된 것을 볼 수 있다.

In [ ]:
#band_1/2 data의 크기는 원래 (5625,)인데, 몇 개 빠져있는 row들이 존재한다. 이를 찾아내기 위한 함수 find_missing_data
def find_missing_data(series, shape):
    count = 0
    missing_list = []
    
    for i, x in enumerate(series):
        if np.shape(series.iloc[i]) != shape:
            missing_list.append(i)
            count += 1
    
    return missing_list, count

In [ ]:
missing_list1, count1 = find_missing_data(train.band_1, (5625,))
print("Count1: ", count1)
print("Missing data: ", missing_list1)

#밑에 나온 값이 band_1 feature에서 shape가 (5625,)가 아닌 값들.

In [ ]:
missing_list2, count2 = find_missing_data(train.band_2, (5625,))
print("Count1: ", count2)
print("Missing data: ", missing_list2)

In [ ]:
missing_list1 == missing_list2
#같은 배열이다. 빠진 것이 나타나는 row가 같다.

In [ ]:
#index에 대해 데이터를 삭제해주는 함수 drop_data
def drop_data(df, index):
    return df.drop(df.index[index])

In [ ]:
train = drop_data(train, missing_list1)
#위의 missing_list1를 이용해 train에서 해당 값을 제거한다.

In [ ]:
train.shape

In [ ]:
print("Number of positive classes: ", len(train[train['is_iceberg'] == 1.0]))
print("Number of negative classes: ", len(train[train['is_iceberg'] == 0.0]))

#### 3가지 정규화 테크닉 사용

In [ ]:
#정규화(정규분포)
def standardise_vector(vector):
    '''standardize vector'''
    standardised_vector = (np.array(vector) - np.mean(vector)) / np.std(vector)
    return standardised_vector.tolist()

In [ ]:
#평균정규화(?)
def mean_normalise_vector(vector):
    '''mean normalize vector'''
    normalised_vector = (np.array(vector) - np.mean(vector)) / (np.max(vector) - np.min(vector))
    return normalised_vector.tolist()

In [ ]:
#최대 최소 정규화
def min_max_scaler(vector, minimum = 0, maximum = 1):
    '''minmaxscaler'''
    X_std  = (np.array(vector) - np.min(vector)) / (np.max(vector) - np.min(vector))
    scaled_vector = X_std * (maximum - minimum) + minimum
    return scaled_vector.tolist()

In [ ]:
#1번 기법 standardise_vector 사용
train['band_1'] = train['band_1'].apply(standardise_vector)
train['band_2'] = train['band_2'].apply(standardise_vector)

In [ ]:
train.head()

In [ ]:
band_1 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
band_2 = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])

In [ ]:
print("Shape of band 1 image:",band_1.shape)
print("Shape of band 2 image:",band_2.shape)

#(1471,5625)가 (1471,75,75)로 변화하였다 -> 2D image로 변환 (추후 Conv2D 연산을 위해 N x Height x Width 꼴로 변환)
#원래는 N x Height x Width x Channel 꼴로 변환해야 하는데, Channel은 밑에서 추가함.

### 2.3 테스트 데이터에 대하여 FE

In [ ]:
#2.2와 같은 작업을 수행한다.
test['inc_angle'] = test['inc_angle'].apply(lambda x: math.radians(x))

In [ ]:
test.inc_angle.head()

In [ ]:
missing_list3, count3 = find_missing_data(test.band_1, (5625,))
print("count: ", count3)
print("missing data: ", missing_list3)

In [ ]:
missing_list4, count4 = find_missing_data(test.band_2, (5625,))
print("count: ", count4)
print("missing data: ", missing_list4)

In [ ]:
test['band_1'] = test['band_1'].apply(standardise_vector)
test['band_2'] = test['band_2'].apply(standardise_vector)

In [ ]:
band_1_test = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
band_2_test = np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])

In [ ]:
print("Shape of test set band 1 image:",band_1_test.shape)
print("Shape of test set band 2 image:",band_2_test.shape)

## 3. Train / Test / Validation Set 나누기

In [ ]:
type(train.is_iceberg)

In [ ]:
#본문 notebook에는 DataFrame.as_matrix() 메서드를 사용하라고 되어있으나, 판다스 0.23.0부터 이 메서드는 더 이상 사용하지 않게 되어 동일한 일을 수행하는 다른 메서드인 DataFrame.to_numpy()를 사용한다.
labels = train.is_iceberg.to_numpy()
angles = train.inc_angle.to_numpy()

In [ ]:
#train 인덱스와 validation 인덱스를 랜덤하게 고른다.
train_indices = np.random.choice(len(labels), round(len(labels)*0.75), replace=False)
validation_indices = np.array(list(set(range(len(labels))) - set(train_indices)))

# 위에서 랜덤하게 고른 인덱스를 사용해 train set 구성
band_1_train = band_1[train_indices]
band_2_train = band_2[train_indices]
angles_train = angles[train_indices]
labels_train = labels[train_indices]

# 위에서 랜덤하게 고른 인덱스를 사용해 validation set 구성
band_1_validation = band_1[validation_indices]
band_2_validation = band_2[validation_indices]
angles_validation = angles[validation_indices]
labels_validation = labels[validation_indices]

# 원본 데이터로부터 test set 구성
band_1_test = band_1_test
band_2_test = band_2_test
angles_test = test.inc_angle.to_numpy()
iD = test.id.to_numpy()

In [ ]:
#모든 데이터를 float datatype으로 변경
band_1_train = band_1_train.astype(np.float32)
band_1_validation = band_1_validation.astype(np.float32)
band_1_test = band_1_test.astype(np.float32)
band_2_train = band_2_train.astype(np.float32)
band_2_validation = band_2_validation.astype(np.float32)
band_2_test = band_2_test.astype(np.float32)
angles_train = angles_train.astype(np.float32)
angles_validation = angles_validation.astype(np.float32)
angles_test = angles_test.astype(np.float32)
labels_train = labels_train.astype(np.float32)
labels_validation = labels_validation.astype(np.float32)
iD = iD.astype(np.str)

In [ ]:
#메모리가 초과되는 불필요한 변수 삭제 <- train을 선언했었는데 왜 del이 안될까???
del(train, test, band_1, band_2)

In [ ]:
#각 데이터들의 shape 출력
print("Shape of band_1_train:",band_1_train.shape)
print("Shape of band_2_train:",band_1_train.shape)
print("Shape of angles_train:",angles_train.shape)
print("Shape of labels_train:",labels_train.shape)
print("Shape of band_1_validation:",band_1_validation.shape)
print("Shape of band_2_validation:",band_2_validation.shape)
print("Shape of angles_validation:",angles_validation.shape)
print("Shape of labels_validation:",labels_validation.shape)
print("Shape of band_1_test:",band_1_test.shape)
print("Shape of band_2_test:",band_2_test.shape)
print("Shape of angles_test:",angles_test.shape)
print("Shape of iD:",iD.shape)

## 4. Train set Augmenting(증강)

### 증강 테크닉을 가져오는 함수 생성

In [ ]:
#이미지 회전
def rotate_image(img, angle = 20):
    
    # rotate image
    original = img.copy()
    
    #2D 이미지를 기하학적으로 변형하기 -> 행렬 변환(강체변환, 유사변환, 선형변환, Affine 등)
    #선형변환은 Vector 공간에서의 이동을 의미.(회전,크기변경, 반전 등. but, 위치변환은 X)
    #Affine 변환은 수평성을 유지하며 선형변환과 이동변환을 포함하는 것.
    #cv2.getRotationMatrix2D(회전 중심 좌표, 반시계 방향의 회전 각도, 추가적인 확대 비율)
    M_rotate = cv2.getRotationMatrix2D((37,37),angle,1)
    
    #cv2.warpAffine(대상 이미지, Affine Matrix, 이미지 크기)
    
    img_new = cv2.warpAffine(img,M_rotate,(75,75))
    
    length_row = 0
    length_column = 0
    boundary_step = 5
    
    for i in range(len(img_new)):
        if img_new[0,i]!=float(0.0):
            length_row = i
            break
    for i in range(len(img_new)):
        if img_new[i,0]!=float(0.0):
            length_column = i
            break
    
    # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
    img_new[:length_column+boundary_step,:length_row+boundary_step] = \
    original[:length_column+boundary_step,:length_row+boundary_step] 
    
    img_new[-(length_row+boundary_step):,:length_column+boundary_step] = \
    original[-(length_row+boundary_step):,:length_column+boundary_step]
    
    img_new[:length_row+boundary_step,-(length_column+boundary_step):] = \
    original[:length_row+boundary_step,-(length_column+boundary_step):]
    
    img_new[-(length_column+boundary_step):,-(length_row+boundary_step):] = \
    original[-(length_column+boundary_step):,-(length_row+boundary_step):]
    
    return img_new

In [ ]:
#수평 방향으로 움직이기(shift)
def translate_horizontal(image, shift_horizontal = 5):

    
    img = image.copy()
    
    shift_vertical = 0; 
    if shift_horizontal<0:
        image_slice = img[:,shift_horizontal:].copy()
    if shift_horizontal>0:
        image_slice = img[:,:shift_horizontal].copy()
    M_translate = np.float32([[1,0,shift_horizontal],[0,1,shift_vertical]])
    img_new = cv2.warpAffine(img,M_translate,(75,75))
    
    # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
    if shift_horizontal<0:
        img_new[:,shift_horizontal:] = image_slice
    if shift_horizontal>0:
        img_new[:,:shift_horizontal] = image_slice
        
    return img_new.reshape(75,75).astype(np.float32)

In [ ]:
#수직으로 움직이기(vertical shift)
def translate_vertical(image, shift_vertical = 5):

    
    img = image.copy()
    
    shift_horizontal = 0;
    if shift_vertical<0:
        image_slice = img[shift_vertical:,:].copy()
    if shift_vertical>0:
        image_slice = img[:shift_vertical,:].copy()
    M_translate = np.float32([[1,0,shift_horizontal],[0,1,shift_vertical]])
    img_new = cv2.warpAffine(img,M_translate,(75,75))
    
    # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
    if shift_vertical<0:
        img_new[shift_vertical:,:] = image_slice
    if shift_vertical>0:
        img_new[:shift_vertical,:] = image_slice
        
    return img_new.reshape(75,75).astype(np.float32)

In [ ]:
#대각선으로 움직이기(diagonal shift)
def translate_positive_diagonal(image, shift_diagonal = 5):

    
    img = image.copy()
    
    if shift_diagonal<0:
        hor_slice = img[shift_diagonal:,:].copy()
        ver_slice = img[:,shift_diagonal:].copy()
    else:
        hor_slice = img[:shift_diagonal,:].copy()
        ver_slice = img[:,:shift_diagonal].copy()
    M_translate = np.float32([[1,0,shift_diagonal],[0,1,shift_diagonal]])
    img_new = cv2.warpAffine(img,M_translate,(75,75))
    
    # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
    if shift_diagonal<0:
        img_new[shift_diagonal:,:] = hor_slice
        img_new[:,shift_diagonal:] = ver_slice
    else:
        img_new[:shift_diagonal,:] = hor_slice
        img_new[:,:shift_diagonal] = ver_slice
    
    return img_new.reshape(75,75).astype(np.float32)

In [ ]:
#대각선으로 움직이기2(diagonal shift) - 음의 방향
def translate_negative_diagonal(image, shift_diagonal = 5):

    
    img = image.copy()
    
    if shift_diagonal<0:
        hor_slice = img[:-shift_diagonal,:].copy()
        ver_slice = img[:,shift_diagonal:].copy()
    if shift_diagonal>0:
        hor_slice = img[-shift_diagonal:,:].copy()
        ver_slice = img[:,:shift_diagonal].copy()
    M_translate = np.float32([[1,0,shift_diagonal],[0,1,-shift_diagonal]])
    img_new = cv2.warpAffine(img,M_translate,(75,75))
    
    # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
    if shift_diagonal<0:
        img_new[:-shift_diagonal,:] = hor_slice
        img_new[:,shift_diagonal:] = ver_slice
    if shift_diagonal>0:
        img_new[-shift_diagonal:,:] = hor_slice
        img_new[:,:shift_diagonal] = ver_slice
        
    return img_new.reshape(75,75).astype(np.float32)

In [ ]:
#이미지 플립
def flip(image, direction = 0):
    
    img = image.copy()
    return cv2.flip(img,direction)

In [ ]:
#이미지 확대
def zoom(image, zoom_shift = 5):

    img = image.copy()

    if zoom_shift>0:
        img_new = cv2.resize(img, (75+zoom_shift*2,75+zoom_shift*2)) 
        img_new = img_new[zoom_shift:-zoom_shift,zoom_shift:-zoom_shift] 
    else:
        zoom_shift *=-1
        
        hor_top = img[:zoom_shift,:]
        hor_bottom =img[-zoom_shift:,:]
        ver_left = img[:,:zoom_shift]
        ver_right = img[:,-zoom_shift:]
        
        img_new = cv2.resize(img, (75-zoom_shift*2,75-zoom_shift*2)) 
        # 제로 패딩
        img_new = cv2.copyMakeBorder(img_new,zoom_shift,zoom_shift,zoom_shift,zoom_shift,
                                     cv2.BORDER_CONSTANT,value=0.0)
        # 이미지 변환 후 남는 부분을 원래 이미지에서 가져온다.
        img_new[:zoom_shift,:] = hor_top
        img_new[-zoom_shift:,:] = hor_bottom
        img_new[:,:zoom_shift] = ver_left
        img_new[:,-zoom_shift:] = ver_right     
        
    return img_new.reshape(75,75).astype(np.float32)

### 4.2 증강된 샘플 살펴보기

In [ ]:
matplotlib.rcParams['figure.figsize'] = (20.0, 14.0)
image = band_1_test[3].copy()
plt.subplot(3, 5, 1)
plt.title("Original Image")
plt.imshow(image)
plt.subplot(3, 5, 2)
generated_image = rotate_image(image,40)
plt.title("Rotation by +ve degree")
plt.imshow(generated_image)
plt.subplot(3, 5, 3)
generated_image = rotate_image(image,-40)
plt.title("Rotation by -ve degree")
plt.imshow(generated_image)
plt.subplot(3, 5, 4)
generated_image = translate_horizontal(image,10)
plt.title("Horizonation translation to right")
plt.imshow(generated_image)
plt.subplot(3, 5, 5)
generated_image = translate_horizontal(image,-10)
plt.title("Horizonation translation to left")
plt.imshow(generated_image)
plt.subplot(3, 5, 6)
generated_image = translate_vertical(image,10)
plt.title("Vertical translation downward")
plt.imshow(generated_image)
plt.subplot(3, 5, 7)
generated_image = translate_vertical(image,-10)
plt.title("Vertical translation upward")
plt.imshow(generated_image)
plt.subplot(3, 5, 8)
generated_image = translate_positive_diagonal(image,10)
plt.title("SE translation")
plt.imshow(generated_image)
plt.subplot(3, 5, 9)
generated_image = translate_positive_diagonal(image,-10)
plt.title("NW translation")
plt.imshow(generated_image)
plt.subplot(3, 5, 10)
generated_image = translate_negative_diagonal(image,10)
plt.title("NE translation")
plt.imshow(generated_image)
plt.subplot(3, 5, 11)
generated_image = translate_negative_diagonal(image,-10)
plt.title("SW translation")
plt.imshow(generated_image)
plt.subplot(3, 5, 12)
generated_image = flip(image,0)
plt.title("Vertical flip")
plt.imshow(generated_image)
plt.subplot(3, 5, 13)
generated_image = flip(image,1)
plt.title("Horizontal flip")
plt.imshow(generated_image)
plt.subplot(3, 5, 14)
generated_image = zoom(image,10)
plt.title("Zoom in")
plt.imshow(generated_image)
plt.subplot(3, 5, 15)
generated_image = zoom(image,-10)
plt.title("Zoom out")
plt.imshow(generated_image)
plt.show()

### 4.3 Train set에 대해 증강 기법 시행

In [ ]:
def augment_data(band1, band2, angles, labels):
    
    '''a function to augment band1 and band2 image'''
    
    # list to store the generated data
    band1_generated = []
    band2_generated = []
    angles_generated = []
    labels_generated = []
    
    # iterate through each point in train set
    for i in range(labels.shape[0]):
        
        # rotate by positive degree
        angle = np.random.randint(5,20)
        band1_generated.append(rotate_image(band1[i],angle)) 
        band2_generated.append(rotate_image(band2[i],angle))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # rotate by negative degree
        angle = np.random.randint(5,20)
        band1_generated.append(rotate_image(band1[i],-angle)) 
        band2_generated.append(rotate_image(band2[i],-angle))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # positive horizontal shift
        shift = np.random.randint(3,7)
        band1_generated.append(translate_horizontal(band1[i],+shift)) 
        band2_generated.append(translate_horizontal(band2[i],+shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # negative horizontal shift
        shift = np.random.randint(3,7) 
        band1_generated.append(translate_horizontal(band1[i],-shift)) 
        band2_generated.append(translate_horizontal(band2[i],-shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # positive vertical shift
        shift = np.random.randint(0,7)  
        band1_generated.append(translate_vertical(band1[i],+shift)) 
        band2_generated.append(translate_vertical(band2[i],+shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # negative vertical shift
        shift = np.random.randint(3,7) 
        band1_generated.append(translate_vertical(band1[i],-shift)) 
        band2_generated.append(translate_vertical(band2[i],-shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # translate along positive diagonal in positive direction
        shift = np.random.randint(3,7)  
        band1_generated.append(translate_positive_diagonal(band1[i],+shift)) 
        band2_generated.append(translate_positive_diagonal(band2[i],+shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # translate along positive diagonal in negative direction
        shift = np.random.randint(3,7)  
        band1_generated.append(translate_positive_diagonal(band1[i],-shift)) 
        band2_generated.append(translate_positive_diagonal(band2[i],-shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # translate along negative diagonal in positive direction
        shift = np.random.randint(3,7)   
        band1_generated.append(translate_negative_diagonal(band1[i],+shift)) 
        band2_generated.append(translate_negative_diagonal(band2[i],+shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # translate along negative diagonal in negative direction
        shift = np.random.randint(3,7)   
        band1_generated.append(translate_negative_diagonal(band1[i],-shift)) 
        band2_generated.append(translate_negative_diagonal(band2[i],-shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # vertical flip
        band1_generated.append(flip(band1[i],0)) 
        band2_generated.append(flip(band2[i],0))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # horizontal flip
        band1_generated.append(flip(band1[i],1)) 
        band2_generated.append(flip(band2[i],1))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # zoom in image
        zoom_shift = np.random.randint(2,5)
        band1_generated.append(zoom(band1[i],zoom_shift)) 
        band2_generated.append(zoom(band2[i],zoom_shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])
        
        # zoom out image
        zoom_shift = np.random.randint(2,5) 
        band1_generated.append(zoom(band1[i],-zoom_shift)) 
        band2_generated.append(zoom(band2[i],-zoom_shift))
        angles_generated.append(angles[i])
        labels_generated.append(labels[i])        
        
    # convert the generated data into numpy array
    band1_generated = np.array(band1_generated)
    band2_generated = np.array(band2_generated)
    angles_generated = np.array(angles_generated)
    labels_generated = np.array(labels_generated)
    
    # concatenate the generated data to original train set
    band1_augmented = np.concatenate((band1, band1_generated),axis=0)
    band2_augmented = np.concatenate((band2, band2_generated),axis=0)
    angles_augmented = np.concatenate((angles, angles_generated),axis=0)
    labels_augmented = np.concatenate((labels, labels_generated),axis=0)
    
    return band1_augmented, band2_augmented, angles_augmented, labels_augmented

In [ ]:
band_1_train, band_2_train, angles_train, labels_train = \
    augment_data(band_1_train, band_2_train, angles_train, labels_train)

In [ ]:
print("Shape of band_1_train:",band_1_train.shape)
print("Shape of band_2_train:",band_2_train.shape)
print("Shape of angles_train:",angles_train.shape)
print("Shape of labels_train:",labels_train.shape)

## 5. band1과 band2데이터를 합치고 이를 3D image로 변환하기

In [ ]:
image_train = np.concatenate([band_1_train[:, :, :, np.newaxis],
                             band_2_train[:, :, :, np.newaxis],
                             ((band_1_train+band_2_train)/2)[:, :, :, np.newaxis]],
                             axis=-1)

In [ ]:
image_validation = np.concatenate([band_1_validation[:, :, :, np.newaxis],
                             band_2_validation[:, :, :, np.newaxis],
                             ((band_1_validation+band_2_validation)/2)[:, :, :, np.newaxis]],
                             axis=-1)

In [ ]:
image_test = np.concatenate([band_1_test[:, :, :, np.newaxis],
                             band_2_test[:, :, :, np.newaxis],
                             ((band_1_test+band_2_test)/2)[:, :, :, np.newaxis]],
                             axis=-1)

In [ ]:
print("Shape of image_train:",image_train.shape)
print("Shape of image_validation:",image_validation.shape)
print("Shape of image_test:",image_test.shape)

## 6. CNN 만들기

In [ ]:
import tensorflow as tf
from tensorflow.python.framework import ops
ops.reset_default_graph()
tf.compat.v1.set_random_seed(tf_rand_seed) #원본 노트북은 TensorFlow 1.x 버전 기준인데, Kaggle Notebook은 TensorFlow 2.x버전이기 때문에, 2.x에서 1.x 코드로 변환했다.
#이하로 compat.v1이 붙는 메서드는 전부 2.x에서 1.x 코드를 돌아가게 하는 방법이다.

In [ ]:
labels_train = pd.get_dummies(labels_train).to_numpy()
labels_validation = pd.get_dummies(labels_validation).to_numpy()

In [ ]:
print("Shape of labels_train:", labels_train.shape)
print("Shape of labels_validation:", labels_validation.shape)

In [ ]:
#이미지 차원 변수 지정
width = 75
height = 75
num_channels = 3
flat = width * height
num_classes = 2

In [ ]:
tf.compat.v1.disable_eager_execution() #TensorFlow 1.x에서 TensorFlow 2.x로 올라갈 때 생기는 오류 방지

#tf.placeholder(TF 1.x) -> tf.compat.v1.placeholder(TF 2.x)

image = tf.compat.v1.placeholder(tf.float32, shape=[None, height, width, num_channels])
y_true = tf.compat.v1.placeholder(tf.int32, shape=[None, num_classes])
keep_prob = tf.compat.v1.placeholder(tf.float32)

### 6.3 딥러닝 레이어를 만들기 위한 함수 생성

In [ ]:
#가중치 텐서 생성
def create_weights(shape):
    return tf.Variable(tf.random.truncated_normal(shape, stddev=0.05))

#바이어스 텐서 생성 
def create_biases(size):
    return tf.Variable(tf.constant(0.05, shape=[size]))

In [ ]:
#컨볼루션 층 만드는 함수
def create_convolutional_layer(input, num_input_channels, conv_filter_size, max_pool_filter_size, num_filters):  

    #컨볼루션 층의 필터 만들기
    weights = create_weights(shape=[conv_filter_size, conv_filter_size, num_input_channels, num_filters])
    
    # 바이어스 값 만들기
    biases = create_biases(num_filters)
    
    # 컨볼루션 층 생성
    layer = tf.nn.conv2d(input=input, filters=weights, strides=[1, 1, 1, 1], padding='SAME')
    
    # 컨볼루션 층에 바이어스 값 더하기
    layer += biases
    
    # ReLU 활성함수 통과
    layer = tf.nn.relu(layer)
    
    # 맥스 풀링을 통해 이미지 사이즈를 절반으로 줄이기
    layer = tf.nn.max_pool2d(input=layer, ksize=[1, max_pool_filter_size, max_pool_filter_size, 1], strides=[1, 2, 2, 1], padding='SAME')
        
    # output layer return
    return layer

In [ ]:
#컨볼루션 층을 지난 것을 flatten하게 해주는 함수
def create_flatten_layer(layer):

    # layer의 size 저장
    layer_shape = layer.get_shape()
    
    #flatten layer가 어떤 shape를 가져야할지 num_features에 저장
    num_features = layer_shape[1:4].num_elements()
    
    
    # flatten layer 만들기
    layer = tf.reshape(layer, [-1, num_features])
    
    return layer

In [ ]:
#fully connected layer 만드는 함수
def create_fc_layer(input, num_inputs, num_outputs, use_relu=True, dropout = False, keep_prob = 0.2):
    

    #가중치와 바이어스값 만들기
    weights = create_weights(shape=[num_inputs, num_outputs])
    biases = create_biases(num_outputs)
    
    # Wx+b 함수 생성 <- 가장 기본 적인 것
    # matrix multiplication between input and weight matrix
    layer = tf.matmul(input, weights) + biases
    
    # ReLU
    if use_relu:
        layer = tf.nn.relu(layer)
        
    # Dropout
    if dropout:        
        layer = tf.nn.dropout(layer, rate=1 - (keep_prob))
    
    return layer

### 6.4 ConvNet의 층 만들기

In [ ]:
# paramters for 1st convolutional layer
conv1_features = 64
conv1_filter_size = 3
max_pool_size1 = 2

# paramters for 2nd convolutional layer
conv2_features = 128
conv2_filter_size = 3
max_pool_size2 = 2

# paramters for 3rd convolutional layer
conv3_features = 128
conv3_filter_size = 3
max_pool_size3 = 2

# paramters for 4th convolutional layer
conv4_features = 64
conv4_filter_size = 3
max_pool_size4 = 2

# number of featuers of 1st fully connected layer
fc_layer_size1 = 512

# number of featuers of 2nd fully connected layer
fc_layer_size2 = 256

In [ ]:
#layer 1
layer_conv1 = create_convolutional_layer(input=image,
                                         num_input_channels= num_channels,
                                         conv_filter_size = conv1_filter_size,
                                         max_pool_filter_size = max_pool_size1,
                                         num_filters = conv1_features)
layer_conv1

In [ ]:
#layer 2
layer_conv2 = create_convolutional_layer(input=layer_conv1,
                                         num_input_channels= conv1_features,
                                         conv_filter_size = conv2_filter_size,
                                         max_pool_filter_size = max_pool_size2,
                                         num_filters = conv2_features)
layer_conv2

In [ ]:
#layer 3
layer_conv3 = create_convolutional_layer(input=layer_conv2,
                                         num_input_channels= conv2_features,
                                         conv_filter_size = conv3_filter_size,
                                         max_pool_filter_size = max_pool_size3,
                                         num_filters = conv3_features)
layer_conv3

In [ ]:
#layer 4
layer_conv4 = create_convolutional_layer(input=layer_conv3,
                                         num_input_channels= conv3_features,
                                         conv_filter_size = conv4_filter_size,
                                         max_pool_filter_size = max_pool_size4,
                                         num_filters = conv4_features)
layer_conv4

In [ ]:
#flatten layer
layer_flat = create_flatten_layer(layer_conv4)
layer_flat

In [ ]:
#layer 5(FC)
layer_fc1 = create_fc_layer(input=layer_flat,
                            num_inputs=layer_flat.get_shape()[1:4].num_elements(),
                            num_outputs=fc_layer_size1,
                            use_relu=True,
                            dropout =True,
                            keep_prob = keep_prob)
layer_fc1

In [ ]:
#layer 6(FC)
layer_fc2 = create_fc_layer(input=layer_fc1,
                            num_inputs=fc_layer_size1,
                            num_outputs=fc_layer_size2,
                            use_relu=True,
                            dropout =True,
                            keep_prob = keep_prob)
layer_fc2

In [ ]:
#layer 7(output)
output_layer = create_fc_layer(input=layer_fc2,
                     num_inputs = fc_layer_size2,
                     num_outputs = num_classes,
                     use_relu=False)
output_layer

### 6.5 예측 & 정확도 메트릭 생성

In [ ]:
# 마지막 출력층에서 사용될 softmax함수
y_pred = tf.nn.softmax(output_layer)

# 예측값
y_pred_cls = tf.argmax(y_pred, axis=1, output_type=tf.int32)

# 실제값
y_true_cls = tf.argmax(y_true, axis=1, output_type=tf.int32)

In [ ]:
# 맞았는지 확인
correct_prediction = tf.equal(y_pred_cls, y_true_cls)

# 정확도 계산 메트릭
accuracy = tf.reduce_mean(input_tensor=tf.cast(correct_prediction, tf.float32))

### 6.6 Optimizer 만들기

In [ ]:
# 크로스엔트로피(loss)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(logits=output_layer,
                                                    labels=y_true)

# mean값 -> loss
loss = tf.reduce_mean(input_tensor=cross_entropy)

In [ ]:
# learning rate of optimizer
learning_rate = (1e-3)*0.30

# train step
train_step = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

## 7. 모델 훈련

In [ ]:
# lists to store the train loss, validation loss, validation accuracy at each iteration
train_loss = []
valid_loss = []
valid_acc = []

# batch size
batch_size = 255
# max iteration
max_iter = 700

In [ ]:
# saver: model 중간 저장
saver = tf.compat.v1.train.Saver(max_to_keep=1)

# variables to store the accuracy, loss, iteration of our best model
best_accuracy = 0
best_loss = 1000000
best_iteration = None

iteration = 0

# create a graph session and optimize under it
with tf.compat.v1.Session() as sess:
    
    # initialize variables
    sess.run(tf.compat.v1.global_variables_initializer())

    # while 57 minutes have not elapsed (to finish before the kernel is killed)
    while (time.time()-t1) < 3420:
        
        # break if max iteration is reached
        if iteration >= max_iter:
            break

        # randomly choosing the indices of the batch 
        rand_index = np.random.choice(labels_train.shape[0], size=batch_size)

        # extract the batch image and labels
        image_rand = image_train[rand_index]
#         angles_rand = angles_train[rand_index]
        labels_rand = labels_train[rand_index]

        # feed dictionary for batch
        feed_dict_batch =  {image: image_rand,
#                             angle: np.transpose([angles_rand]),
                            y_true: labels_rand,
                            keep_prob: 0.7}
        # feed dictionary for train
        feed_dict_train =  {image: image_rand,
#                             angle: np.transpose([angles_rand]),
                            y_true: labels_rand,
                            keep_prob: 1.0}
        # feed dictionary for validation
        feed_dict_validation =  {image: image_validation,
#                                  angle: np.transpose([angles_validation]),
                                 y_true: labels_validation,
                                 keep_prob: 1.0}
        
        # 최적화 작업 종료
        sess.run(train_step, feed_dict=feed_dict_batch)

        # calculate temporary train loss and append it to the designated list
        temp_train_loss = loss.eval(session=sess, feed_dict=feed_dict_train)
        train_loss.append(temp_train_loss)
        # calculate temporary validation loss and append it to the designated list
        temp_validation_loss = loss.eval(session=sess, feed_dict=feed_dict_validation)
        valid_loss.append(temp_validation_loss)
        # calculate temporary validation accuracy and append it to the designated list
        temp_validation_accuracy = accuracy.eval(session=sess, feed_dict=feed_dict_validation)
        valid_acc.append(temp_validation_accuracy)
        
        #valid loss가 best loss와 같고 accuracy가 best accuracy보다 좋으면 best model의 파라미터 갱신 후 저장
        if (temp_validation_loss == best_loss) and (temp_validation_accuracy > best_accuracy):
            best_accuracy = temp_validation_accuracy
            best_loss = temp_validation_loss
            best_iteration = iteration           
            saver.save(sess, './my-model', global_step = best_iteration)
        
        # valid accuray가 더 좋으면 best accuracy 갱신
        if temp_validation_accuracy > best_accuracy:
            best_accuracy = temp_validation_accuracy
        
        # valid loss가 best loss 보다 적으면 이를 best_loss에 저장하고 model 저장.
        # update the parameters of the best model and save the model
        if temp_validation_loss < best_loss:
            best_loss = temp_validation_loss
            best_iteration = iteration          
            saver.save(sess, './my-model', global_step = best_iteration)

        # 학습할 때마다 출력할 문장
        print("iterations:",iteration,
              "| train_loss:", temp_train_loss,
              "| validation_loss:", temp_validation_loss,
              "| valid_accuracy:", temp_validation_accuracy)
        
        # iteration 값 증가
        iteration = iteration+1

In [ ]:
# delete unnecessary variables out of memory
del(image_train, image_validation, angles_train, angles_validation, labels_train, labels_validation)

In [ ]:
with tf.Session() as sess:    
    
    # restore the best model
    model_path = "./"+"my-model-"+str(best_iteration)
    saver.restore(sess, model_path)
    
    # break the test set into k folds other wise kernel will be out of memory
    n = len(iD)
    k = 12
    step = n//k
    
    # array to store the prediction
    preds = np.array([])

    # iterate through each fold
    for i in range(k):

        # start and end indices of the fold
        start = (step*i)
        end = (step*(i+1)) 
    
        # feed dictionary for the fold
        feed_dict_test =  {image: image_test[start:end],
#                            angle: np.transpose([angles_test[start:end]]),
                           keep_prob: 1.0}

        # evaluate predictions of the fold
        fold_preds = y_pred.eval(session=sess, feed_dict = feed_dict_test)[:,1]
        # append the predictions of the fold to the designated array
        preds = np.append(preds, fold_preds)
    
    # save the submission csv file
    submission_path = "./submission.csv"
    submission = pd.DataFrame({"id": iD, "is_iceberg": preds})
    submission.to_csv(submission_path, header = True, index=False)
    
    # save the csv file containing performance metrics of the best model 
    results = pd.DataFrame([int(best_iteration),train_loss[best_iteration],
                            valid_loss[best_iteration], valid_acc[best_iteration]],
                           index=["iteration", "train loss", "valid loss", "accuracy"],
                           columns = ["results"])    
    results_path = "./results.csv"    
    results.to_csv(results_path, header = True, index=True)

In [ ]:
plt.figure(figsize=(16, 8), dpi= 80, facecolor='w', edgecolor='k')
iterations = list(range(1,iteration+1))
plt.plot(iterations, train_loss, label = "train loss")
plt.plot(iterations, valid_loss, label = "valid loss")
plt.title("Loss")
plt.xlabel("iter")
plt.ylabel("loss")
plt.legend()
plt.grid()
plt.show()